# Merge with conditions

Using the raw data from the **product_prices_cleaned.csv** file, perform the task of finding out how many times a lower price has been quoted for a given product, province, month historically. To do this, perform the following steps:

1. merge the table with itself. What type of join should be used?
1. filter the data to find earlier years and the values smaller than the one from current year in a given province.
1. group the data accordingly.

Which product(s) had the most such occurrences?

> See what happens to the column names when the columns are not used as a merge condition, but have the same names

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../../01_Data/product_prices_cleaned.csv', sep=';')
df = df[['province', 'product', 'value', 'date']]
print(df.shape)
df.head()

(128503, 4)


,province,product,value,date
0,SUBCARPATHIA,pork ham cooked - per 1kg,21.37,2013-3
1,ŁÓDŹ,bread - per 1kg,NaN,2018-2
2,KUYAVIA-POMERANIA,barley groats sausage - per 1kg,3.55,2019-12
3,LOWER SILESIA,dressed chickens - per 1kg,6.14,2019-2
4,WARMIA-MASURIA,Italian head cheese - per 1kg,5.63,2002-3


In [4]:
%%time
# Merge the table with itself
# See what happens to the column names when the columns are not used as a merge condition, but have the same names

merged_df = pd.merge(df, df, on=['province', 'product'], how='left')

# merged_df = pd.merge(left=df, right=df, how='cross')

print(merged_df.shape)
merged_df.head()

(32378489, 6)
CPU times: user 1.85 s, sys: 1.38 s, total: 3.23 s
Wall time: 4.1 s


,province,product,value_x,date_x,value_y,date_y
0,SUBCARPATHIA,pork ham cooked - per 1kg,21.37,2013-3,21.37,2013-3
1,SUBCARPATHIA,pork ham cooked - per 1kg,21.37,2013-3,15.76,2007-1
2,SUBCARPATHIA,pork ham cooked - per 1kg,21.37,2013-3,20.88,2019-10
3,SUBCARPATHIA,pork ham cooked - per 1kg,21.37,2013-3,16.26,2010-4
4,SUBCARPATHIA,pork ham cooked - per 1kg,21.37,2013-3,17.52,2011-5


In [7]:
# Filter the data to find earlier years and the values smaller than the one from current year in a given province.

filtered_df = merged_df.loc[
    (merged_df['value_y'] < merged_df['value_x'])
#     & (merged_df['date_y'] < merged_df['date_x'])
    & (merged_df['date_x'] == merged_df['date_x'].max()) 
    
]

In [9]:
# Grouping the data to find occurrences
grouped_data = filtered_df.groupby(['product', 'province']).size().reset_index(name='occurrences')

grouped_data

,product,province,occurrences
0,30% tomato concentrate - per 1kg,GREATER POLAND,2
1,30% tomato concentrate - per 1kg,LESSER POLAND,66
2,30% tomato concentrate - per 1kg,LOWER SILESIA,192
3,30% tomato concentrate - per 1kg,LUBUSZ,247
4,30% tomato concentrate - per 1kg,MASOVIA,27
...,...,...,...
387,whole pickled cucumbers 0.9l - per 1pc.,POMERANIA,79
388,whole pickled cucumbers 0.9l - per 1pc.,SILESIA,6
389,whole pickled cucumbers 0.9l - per 1pc.,SUBCARPATHIA,230
390,whole pickled cucumbers 0.9l - per 1pc.,WARMIA-MASURIA,240


In [10]:
# Finding which product(s) had the most such occurrences
most_occurrences = grouped_data.groupby('product')['occurrences'].sum().sort_values(ascending=False)

most_occurrences.head()


product
plain mixed bread (wheat-rye) - per 1kg     3720
pork ham cooked - per 1kg                   3363
pork belly cooked - per 1kg                 3334
pork meat with bone (shoulder) - per 1kg    2609
Italian head cheese - per 1kg               2540
Name: occurrences, dtype: int64